In [1]:
import pandas as pd
import sqlite3
from sklearn.preprocessing import LabelEncoder, StandardScaler
from joblib import load

In [2]:
con = sqlite3.connect('data/database.db')

df = pd.read_sql_query(' \
    SELECT loanDev_view.id as loanId, account_view.id as accountId, ownerSex, ownerBirthday, ownerCardType, frequency, account_view.date AS accountCreationDate, isShared, balance, loanDev_view.date AS loanDate, duration as loanDuration, payments as loanPayments, amount as loanAmount, ratio, districtName, region, nInhabitants, nMunicipalitiesSub499Inhabitants, nMunicipalities500to1999Inhabitants, nMunicipalities2000to9999Inhabitants, nMunicipalitiesOver10000Inhabitants, nCities, urbanInhabitantsRatio, averageSalary, unemploymentRate95, unemploymentRate96, nEnterpreneursPer1000Inhabitants, commitedCrimes95, commitedCrimes96, \
    medianAmount, \
    sumAllTransactions, \
    insurancePaymentsCount, \
    insurancePaymentsAverage, \
    timesIntoNegativeBalance, \
    numTransactions, \
    numTransactionsNegBalance, \
    numExternalBankTransactions, \
    withdrawalCount, \
    cashWithdrawalCount,  \
    withdrawalAnyMethodCount,  \
    creditCount \
    maxWithdrawal, \
    maxCredit, \
    maxTransactionAmountDistance, \
    sumSanctionInterest, \
    avgSanctionInterest,  \
    hasStableIncome,    \
    status \
    FROM loanDev_view \
    JOIN account_view ON account_view.id = loanDev_view.accountId \
    JOIN owners_sex_birthday_view ON owners_sex_birthday_view.accountId = account_view.id \
    LEFT OUTER JOIN owners_card_type ON owners_card_type.accountId = account_view.id \
    JOIN district_view ON district_view.id = account_view.districtId;'
, con)

trans_df = pd.read_sql_query("SELECT * FROM ( \
    select id, accountId, amount as credit, null as withdrawal from transDev where type='credit' \
    UNION \
    select id, accountId, null as credit, amount as withdrawal from transDev where type='withdrawal' or type='withdrawal in cash' \
    );"
, con)

trans_df['amount'] = trans_df['credit'] if trans_df['withdrawal'].isna else -trans_df['withdrawal']

trans_df = trans_df.groupby(['accountId']).apply(lambda x: x.quantile(0.75) - x.quantile(0.25)).drop('accountId', axis=1)
trans_df = df.join(trans_df, on='accountId')
df['transactionAmountIQR'] = trans_df['amount']
df['transactionAmountIQR'] =df['transactionAmountIQR'].fillna(df['transactionAmountIQR'].mean())
df['maxTransactionAmountDistance'] = df['maxTransactionAmountDistance'].fillna(df['maxTransactionAmountDistance'].mean())

df = df.drop('accountId', axis=1)
df.to_csv('data/unified_data.csv', index=False)
df.head()

A


,loanId,ownerSex,ownerBirthday,ownerCardType,frequency,accountCreationDate,isShared,balance,loanDate,loanDuration,...,cashWithdrawalCount,withdrawalAnyMethodCount,maxWithdrawal,maxCredit,maxTransactionAmountDistance,sumSanctionInterest,avgSanctionInterest,hasStableIncome,status,transactionAmountIQR
0,5895,1,620601,None,monthly issuance,951009,0,49548.5,970103,60,...,0,59,30,0.0,77973.136364,0.0,0.0,0,0,24192.175
1,7122,0,490121,None,monthly issuance,950902,0,11565.4,970104,36,...,1,49,29,0.0,77973.136364,4.6,4.6,0,0,12763.400
2,6173,0,390427,None,issuance after transaction,950419,0,45754.0,970108,48,...,7,133,43,0.0,77973.136364,0.0,0.0,1,0,28871.000
3,6142,0,430929,None,monthly issuance,960316,1,38913.4,970121,60,...,5,50,21,0.0,77973.136364,0.0,0.0,1,0,22014.200
4,5358,1,391206,None,monthly issuance,950604,0,16514.3,970121,12,...,0,62,52,36463.5,70063.500000,0.0,0.0,0,0,4580.000


In [3]:
cc95 = pd.to_numeric(df['commitedCrimes95'], errors='coerce')
diff = df['commitedCrimes96'] - cc95
averageDiff = diff.mean()
cc95[cc95.isna()] = df['commitedCrimes96'][cc95.isna()] - averageDiff
df['commitedCrimes95'] = cc95.astype(int)

ur95 = pd.to_numeric(df['unemploymentRate95'], errors='coerce')
diff = df['unemploymentRate96'] - ur95
averageDiff = diff.mean()
ur95[ur95.isna()] = df['unemploymentRate96'][ur95.isna()] - averageDiff
df['unemploymentRate95'] = ur95

df['hasCard'] = df['ownerCardType'].apply(lambda x: False if pd.isna(x) else True)
df = df.drop('ownerCardType', axis=1)

df.to_csv('data/clean_data1.csv', index=False)

In [10]:
def convert_to_unix_timestamp(df, attribute):
    df[attribute] = df[attribute].apply(lambda x: "19" + str(x))
    df[attribute] = pd.to_datetime(df[attribute], format="%Y%m%d").map(pd.Timestamp.timestamp)
    return df

df = pd.read_csv('data/clean_data1.csv')

for attribute in ['ownerBirthday', 'accountCreationDate', 'loanDate']:
    df = convert_to_unix_timestamp(df, attribute)

df = pd.get_dummies(df, columns=['frequency', 'region'], drop_first=True)

df["districtName"] = LabelEncoder().fit_transform(df["districtName"])

# Roughly scale dataset
scaler = StandardScaler().fit(df)
scaled_df = pd.DataFrame(data=scaler.transform(df), columns=df.columns)
scaled_df['status'] = df['status']
scaled_df['loanId'] = df['loanId']

scaled_df.to_csv('data/prepared_data.csv', index=False)
df = scaled_df

In [11]:
ids, df = df['loanId'], df[['isShared', 'balance', 'loanPayments', 'sumAllTransactions',   
       'timesIntoNegativeBalance', 'numTransactionsNegBalance',
       'numExternalBankTransactions', 'withdrawalCount', 'hasStableIncome',
       'transactionAmountIQR']]

In [12]:
df.head()

,isShared,balance,loanPayments,sumAllTransactions,timesIntoNegativeBalance,numTransactionsNegBalance,numExternalBankTransactions,withdrawalCount,hasStableIncome,transactionAmountIQR
0,-0.496466,0.299019,-1.190541,0.324807,-0.171592,-0.123387,0.143759,0.316935,-0.667007,0.139654
1,-0.496466,-1.394891,1.347572,-1.403224,3.203052,1.932095,0.369189,-0.043504,-0.667007,-0.885471
2,-0.496466,0.129798,0.276232,0.152168,-0.171592,-0.123387,3.637916,2.512338,1.499235,0.559329
3,2.014235,-0.175268,-0.236848,-0.159035,-0.171592,-0.123387,0.989120,-0.141806,1.499235,-0.055704
4,-0.496466,-1.174188,-0.455141,-1.292726,-0.171592,-0.123387,-0.025313,0.415237,-0.667007,-1.619496


In [13]:
model = load('data/model.joblib')

In [21]:
binary_predictions = model.predict(df)
print(binary_predictions)
predictions = model.predict_proba(df)
print(predictions[:,1])

[0 1 0 0 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1
 0 1 1 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 1 0 1 0 1 1
 1 0 0 1 0 1 1 0 0 0 0 1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0
 0 1 0 0 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 1 0 0 0 1 0 0 1 0 1 1 0 0 1 0 0 1 1
 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 0 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 0 1
 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1]
[0.29 0.99 0.3  0.24 0.63 0.27 0.51 0.29 0.43 0.5  0.75 0.79 0.88 0.97
 0.78 0.48 0.51 0.66 0.27 0.43 0.48 0.58 0.97 0.51 0.3  0.52 0.54 0.85
 0.5  0.49 0.4  0.48 0.42 0.29 0.35 0.29 0.61 0.37 0.73 0.68 0.62 0.4
 0.27 0.7  0.34 0.27 0.48 0.24 0.36 0.4  1.   0.5  0.89 0.6  0.32 0.3

In [22]:
print(f"Ratio:{sum(binary_predictions)/len(binary_predictions)}")

Ratio:0.3700564971751412


In [19]:
with open("submission.csv", "w") as file:
    file.write('Id,Predicted\n')
    for i, pred in zip(ids, predictions[:,1]):
        file.write(f"{i},{pred}\n")